## Efficient Computation and Storage of Spectrograms

In this notebook we explore ways to compute and store efficiently spectrograms from long streams of audio data. Our approach aims to handle the computation in chunks without simultaneously loading the whole dataset, and use storage formats which allow the extraction of subsets  of the data without the reading the full file. We investigate the use of `xarray` and `dask` libraries and the `netcdf` format. The test audio dataset comes from the Ocean Observatories Initiative hydrophone data.  We have a big `.mseed` audio file which stores 2 hours data. The file was created by the hydrophone team during [Cabled Array Hackweek](https://oceanhackweek.github.io/CAHW2018_website/), when we first encountered the challenge of handling spectrograms of long audio streams.

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
# xarray is a library for labeled multidimensional arrays which supports chunking
import xarray as xr

In [ ]:
# obspy is a library to read .mseed audio streams (developed in the seismology community)
import obspy

In [ ]:
obspy.__version__

In [ ]:
# visualization packages
import matplotlib.pyplot as plt
from matplotlib import mlab
import pylab as pl
%matplotlib inline

In [ ]:
# ~1GB file of long hydrophone audio stream 
# (this file was extracted and created during Cabled Array Hackweek)
# need to store it somewhere publicly
merged_sounds = obspy.read('../data/merged_hydrophone.mseed')

In [ ]:
merged_sounds = merged_sounds.merge()

In [ ]:
# calculate number of frames in clip
secs = 5
freq = 64000
framesPerClip= secs*freq

In [ ]:
params = {'NFFT':1024, 'Fs':64000, 'noverlap':5}
P, freqs, bins = mlab.specgram(merged_sounds[0].data[:framesPerClip], **params)
spec_dim = P.shape

In [ ]:
# function to plot the spectrogram
def PlotSpecgram(P, freqs, bins):
    """Spectrogram"""
    Z = np.flipud(P) # flip rows so that top goes to bottom, bottom to top, etc.
    xextent = 0, np.amax(bins)
    xmin, xmax = xextent
    extent = xmin, xmax, freqs[0], freqs[-1]

    im = pl.imshow(np.log(Z[:,:]), extent=extent,cmap = 'plasma')
    pl.axis('auto')
    pl.xlim([0.0, bins[-1]])
    pl.ylim([0, freqs[-1]])

In [ ]:
#PlotSpecgram(spec, freqs, times)

In [ ]:
# specify window length for parallel processing
# a small overlap will ensure there are no gaps in the values
# but need to make sure there are no conflicts at the boundaries
# eventually each window becomes a chunk
# the size of the window also affects the choice of nfft in the short term fourier transform

window_length = 5 
step = 5
windows = merged_sounds[0].slide(5,5)

In [ ]:
st = obspy.Stream([windowed_st for windowed_st in merged_sounds[0].slide(window_length=10.0, step=10.0)])

In [ ]:
from scipy import signal

In [ ]:
nfft = 4096
noverlap = 5
fs = 64000

In [ ]:
# let's calculate the spectrogram of the first window in the stream (st[0])
freqs, times, spec = signal.spectrogram(st[0].data, fs=fs, nfft=nfft, noverlap=noverlap)

In [ ]:
import datetime

In [ ]:
# start_time
start_time = datetime.datetime.strptime(str(st[0].stats.starttime),'%Y-%m-%dT%H:%M:%S.%fZ')
st[0].stats.starttime

In [ ]:
(start_time+datetime.timedelta(0,1)).strftime("%Y-%m-%d %H:%M:%S")

In [ ]:
# convert times to UTC
UTC_times = [datetime.timedelta(0,sec)+start_time for sec in times]

In [ ]:
# create an xarray with correct time and frequency labels
xr_spec = xr.DataArray(spec,dims = ('freq','time'),coords = {'freq':freqs,'time':UTC_times})

In [ ]:
# chunk it: the chunk is equal to the window: i.e. the whole window is a chunk by itself
xr_spec.chunk(xr_spec.shape)

In [ ]:
# now we can use the built-in xarray function to plot the spectrogram with the correct labels
# we apply the log function for better visibility
plt.figure(figsize=(10,5))
np.log(xr_spec).plot()

In [ ]:
total_curve = xr_spec.sel(freq=slice(0,200)).sum(axis = 0)

In [ ]:
plt.figure(figsize=(10,5))
total_curve.plot()

In [ ]:
# spectrogram function definition
def get_spec(trace, fs, nfft, noverlap, outfile=False):
    """
    create spectrogram from obspy trace to xarray format
    
    Inputs
    ------
        trace: audio signal in obspy trace format
        fs, nfft, noverlap: stft parameters
        outfile: if True, the spectrogram is written into a sequence of netcdf files
        
        Note: the name of the files is merged_hydrophone_[start_time].nc
        TODO: more custom file naming
        
    Outputs
    -------
        xr_spec: log of spectrogram in xarray format
       
    """
    
    # spectrogram calculation
    freqs, times, spec = signal.spectrogram(trace.data, fs=fs, nfft=nfft, noverlap=noverlap)
    
    # creating time stamps
    start_time = datetime.datetime.strptime(str(trace.stats.starttime),'%Y-%m-%dT%H:%M:%S.%fZ')
    UTC_times = [datetime.timedelta(0,sec)+start_time for sec in times]
    xr_spec = xr.DataArray(np.log(spec),dims = ('freq','time'),coords = {'freq':freqs,'time':UTC_times})
    
    # chunking
    xr_spec = xr_spec.chunk(xr_spec.shape)
    
    # write output file
    if outfile:
        xr_spec.to_netcdf('merged_hydrophone_'+str(start_time)+'.nc')
        
    return(xr_spec)

We will use the `dask.delayed` functionality. We will wrap the `get_spec` function and will apply it in parrallel to all windows in the stream.

In [ ]:
from dask.delayed import delayed

In [ ]:
get_spec_delayed = delayed(get_spec)

In [ ]:
dfs = [get_spec_delayed(tr,fs,nfft,noverlap) for tr in st]

In [ ]:
%%time
import dask
results = dask.compute(*dfs)

In [ ]:
# TODO: understand why I get 0's in the spectrograms

In [ ]:
results[0]

In [ ]:
# concatenating the list of outputs
xr_spec_combined = xr.concat(results, dim='time')
    

In [ ]:
# xr_spec_combined_sorted = xr_spec_combined.sortby('time')

In [ ]:
# xr_spec_combined_sorted.shape

Let's look at a slice of the spectrogram. We can use the `xarray`'s `sel` function.

In [ ]:
sliced = xr_spec_combined.sel(time=slice('2017-08-21T09:02:10','2017-08-21T09:02:20'))

In [ ]:
sliced

In [ ]:
sliced.time[0]

In [ ]:
%%time
plt.figure(figsize=(10,5))
sliced.sel(freq=slice(0,500)).plot()

In [ ]:
sliced.sel(freq=slice(0,500)).imshow()

#### Storing and opening spectrograms in NetCDF format

*Single File*

In [ ]:
# Storing
#
# if a filename already exists,
# we need to specify the mode with which we want to write up the file: 'w' or 'a'

xr_spec_combined.to_netcdf('spectrogram.nc',mode='w')

In [ ]:
# Opening

spec_opened = xr.open_dataarray('../data/spectrogram.nc')
spec_opened.chunk(results[0].shape)

In [ ]:
# chunks did not get preserved
spec_opened

In [ ]:
# sort by time: plotting is complaining xarray is not sorted, 
# so trying to sort it but crashing the kernel
spec_opened = spec_opened.sortby('time')

In [ ]:
spec_opened.plot()

*Multiple Files*

In [ ]:
#!ls

In [ ]:
# cleaning up unnecessary files
#!find . -name "2017*.nc" -exec rm {} \;

In [ ]:
# Working with multiple files: they automatically load into a chunked format
res = xr.open_mfdataset('../data/spectrogram.nc')

In [ ]:
res.sel(time=slice('2017-08-21T09:02','2017-08-21T09:04'))

In [ ]:
res.plot()

In [ ]:
# Working with multiple files: they automatically load into a chunked format
res = xr.open_mfdataset('../data/merge*.nc', chunks = {'freq':2049,'time':7647})